In [4]:
"""
# 🚀 Detector de Objetos basado en Clasificador Pre-entrenado

Este script Python implementa un sistema para clasificar objetos en una secuencia de imágenes (fotogramas de vídeo).
Utiliza una red neuronal convolucional pre-entrenada (ResNet50 en ImageNet) para identificar el objeto más probable
en cada fotograma y reporta el grado de confianza de la clasificación.

## Funcionalidades:
1.  **Descarga** de imágenes de ejemplo para simular fotogramas de vídeo.
2.  **Aumento de datos** simple para crear variaciones de los fotogramas.
3.  Carga de un **clasificador pre-entrenado** (ResNet50 en ImageNet).
4.  Realización de **inferencia** sobre el directorio de fotogramas.
5.  Generación de un **DataFrame de Pandas** con los resultados de la clasificación.

## Requisitos:
Asegúrate de tener instaladas las siguientes librerías:
-   `torch`
-   `torchvision`
-   `requests`
-   `Pillow` (PIL)
-   `pandas`

Puedes instalarlas con pip:
`pip install torch torchvision torchaudio requests Pillow pandas`
"""

import os
import requests
import shutil
import random
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image, ImageEnhance
import pandas as pd

"""
# ⚙️ PARTE 1 - DESCARGA DE IMÁGENES DE MUESTRA

En esta sección, descargamos un conjunto de imágenes desde URLs públicas.
Estas imágenes servirán como los "fotogramas" de un vídeo que el modelo procesará.
Se guardarán en un directorio local para su posterior análisis.
"""

# URLs públicas de ejemplo (puedes añadir más si quieres)
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/4/47/American_Eskimo_Dog.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/6/68/Orange_tabby_cat_sitting_on_fallen_leaves.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/1/18/Dog_Breeds.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/c/c1/Six_weeks_old_cat_%28aka%29.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/5/50/Vd-ej204-vuosaari.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/a/a6/Pink_floyd_car.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/d/d5/2019_Toyota_Corolla_Icon_Tech_VVT-i_1.8_Front.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/4/4f/Apple_MacBook_Air_M1.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/8/88/Banana-Single.jpg"
]

save_dir = "sample_frames"
os.makedirs(save_dir, exist_ok=True)

print("📥 DESCARGANDO IMÁGENES DE MUESTRA")
for i, url in enumerate(image_urls):
    filename = f"frame_{i+1:04d}.jpg"
    filepath = os.path.join(save_dir, filename)
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filepath, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            print(f"✅ Descargada: {filename}")
        else:
            print(f"❌ Error descargando {url}: {response.status_code} {response.reason}")
    except Exception as e:
        print(f"❌ Error descargando {url}: {e}")

"""
# 🔁 PARTE 2 - AUMENTO DE DATOS (Data Augmentation)

Para simular una mayor variedad de fotogramas o condiciones de vídeo,
aplicamos técnicas simples de aumento de datos a las imágenes descargadas.
Esto incluye ajustes de brillo, contraste y volteo horizontal.
"""

def augment_image(image):
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.7, 1.3))
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(random.uniform(0.7, 1.3))
    return image.transpose(Image.FLIP_LEFT_RIGHT)

# Crear variaciones
existing_files = sorted([f for f in os.listdir(save_dir) if f.endswith(".jpg")])
for i, filename in enumerate(existing_files):
    path = os.path.join(save_dir, filename)
    image = Image.open(path).convert("RGB")
    variation = augment_image(image)
    # Se añade un offset de 1000 para que los nombres de archivo sean únicos y se puedan ordenar fácilmente
    new_filename = f"frame_{1000 + i:04d}.jpg"
    variation.save(os.path.join(save_dir, new_filename))
    print(f"✅ Creada variación: {new_filename}")

"""
# 🤖 PARTE 3 - CLASIFICADOR PREENTRENADO (IMAGENET)

Aquí configuramos la red neuronal convolucional pre-entrenada que usaremos para la clasificación.
Se utiliza ResNet50, un modelo popular entrenado en el conjunto de datos ImageNet,
capaz de clasificar 1000 categorías de objetos.

Se definen las transformaciones necesarias para preparar las imágenes para el modelo.
"""

# Transformaciones: Redimensionar, convertir a tensor y normalizar (valores estándar de ImageNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Modelo: ResNet50 pre-entrenado
model = models.resnet50(pretrained=True)
model.eval() # Poner el modelo en modo de evaluación (importante para inferencia)

# Clases de ImageNet: Descargamos la lista de las 1000 clases de ImageNet
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
labels_path = "imagenet_classes.txt"
if not os.path.exists(labels_path):
    print("⬇️ Descargando etiquetas de ImageNet...")
    r = requests.get(LABELS_URL)
    with open(labels_path, 'w') as f:
        f.write(r.text)
    print("✅ Etiquetas descargadas.")

with open(labels_path) as f:
    classes = [line.strip() for line in f.readlines()]

"""
# 🧠 PARTE 4 - INFERENCIA

Esta sección define la función principal que procesa todas las imágenes
en el directorio especificado, realiza la clasificación con el modelo
pre-entrenado y recopila los resultados.
"""

def classify_images_in_folder(folder_path):
    """
    Clasifica las imágenes en una carpeta dada utilizando el modelo pre-entrenado.

    Args:
        folder_path (str): Ruta al directorio que contiene los fotogramas del vídeo.

    Returns:
        pd.DataFrame: Un DataFrame con el número de fotograma, la clase detectada
                      y la confianza de la clasificación.
    """
    results = []
    # Obtener y ordenar los archivos para mantener el orden de los fotogramas
    files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg')])

    print(f"\n🚀 Iniciando inferencia en {len(files)} fotogramas...")
    for filename in files:
        try:
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('RGB')
            # Aplicar transformaciones y añadir una dimensión de batch
            input_tensor = transform(image).unsqueeze(0)

            with torch.no_grad(): # Desactivar el cálculo de gradientes para la inferencia
                output = model(input_tensor)
                # Convertir logits a probabilidades
                probs = torch.nn.functional.softmax(output[0], dim=0)
                # Obtener la clase con la probabilidad más alta
                confidence, pred_class = torch.max(probs, 0)

            # Extraer el número de fotograma del nombre del archivo
            frame_number = int(filename.split('_')[-1].split('.')[0])
            class_name = classes[pred_class] # Mapear el índice a la clase legible

            results.append({
                "fotograma": frame_number,
                "clase_detectada": class_name,
                "confianza": float(confidence) # Convertir a float estándar de Python
            })
        except Exception as e:
            print(f"⚠️ Error procesando {filename}: {e}")
    
    # Crear y ordenar el DataFrame de resultados
    df = pd.DataFrame(results).sort_values(by="fotograma").reset_index(drop=True)
    return df

"""
# 📊 PARTE 5 - RESULTADOS Y VISUALIZACIÓN

Ejecutamos la función de inferencia y mostramos los primeros resultados
en formato de DataFrame de Pandas.
"""

df_resultados = classify_images_in_folder(save_dir)
print("\n📊 RESULTADOS DE CLASIFICACIÓN")
print(df_resultados.head(10)) # Muestra los primeros 10 resultados del DataFrame

"""

📥 DESCARGANDO IMÁGENES DE MUESTRA
❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/4/47/American_Eskimo_Dog.jpg: 403 Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy
❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/6/68/Orange_tabby_cat_sitting_on_fallen_leaves.jpg: 404 Not Found
✅ Descargada: frame_0003.jpg
❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/c/c1/Six_weeks_old_cat_%28aka%29.jpg: 403 Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy
❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/5/50/Vd-ej204-vuosaari.jpg: 404 Not Found
❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/a/a6/Pink_floyd_car.jpg: 404 Not Found
❌ Error descargando https://upload.wikimedia.org/wikipedia/commons/d/d5/2019_Toyota_Corolla_Icon_Tech_VVT-i_1.8_Front.jpg: 403 Forbidden. Please comply with the User-Agent p